# Estudio Junio - Julio - Agosto (JJA)
## Pablo Lavín
## TFM - Master Ciencia de Datos (UC-UIMP)
## Beca JAE Intro ICU 2025
## Septiembre 2025 - Junio 2026

En este notebook se analiza el desempeño del modelo ECMWF frente a ERA5-Land durante el periodo 1981–2022 (JJA).

Con el objetivo de mejorar la representatividad del modelo respecto a los datos observacionales, se realiza una corrección del sesgo de la temperatura máxima del modelo ECMWF.

Posteriormente los datos de temperatura se convierten en índices, en este caso, un índice de temperatura máxima (número de días que se supera una temperatura umbral).

Se incluyen métricas de sesgo, correlación, RMSE, ratio de varianzas y ROCSS.

## Configuración

In [1]:
# Cargamos paquetes
source("../../scripts/setup_libraries.R")
source("../../scripts/load_bc_functions.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: udunits2

udunits system database read from /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/share/udunits/udunits2.xml

convertR version 0.3.0 (2025-07-31) is loaded


Development version may have an unexpected behaviour

  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R


Attaching package: ‘convertR’


The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs, tdps2hurs


visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: climdex.pcic

Loading required package: PCICt

climate4R.climdex version 0

In [2]:
# Region de estudio

lon = c(-10, 5)
lat = c(35, 44)

# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

In [3]:
# Cargo datos
tmx_modelo_0 = readRDS("../../data/jja_data/tmx_modelo_jja_0.rds")
tmx_modelo_1 = readRDS("../../data/jja_data/tmx_modelo_jja_1.rds")
tmx_modelo_2 = readRDS("../../data/jja_data/tmx_modelo_jja_2.rds")
tmx_modelo_3 = readRDS("../../data/jja_data/tmx_modelo_jja_3.rds")

tmx_obs_cel = readRDS("../../data/jja_data/tmx_obs_cel_jja.rds")

## biasCorrection (bC)

In [ ]:
folds_list = lapply(1981:2022, function(x) x)
                    
bc_0 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_0,
    newdata = tmx_modelo_0,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_1 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_1,
    newdata = tmx_modelo_1,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_2 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_2,
    newdata = tmx_modelo_2,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_3 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_3,
    newdata = tmx_modelo_3,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

In [ ]:
saveRDS(bc_0, file = "bc_jja_0.rds")
saveRDS(bc_1, file = "bc_jja_1.rds")
saveRDS(bc_2, file = "bc_jja_2.rds")
saveRDS(bc_3, file = "bc_jja_3.rds")

In [4]:
bc_0 = readRDS("../../data/jja_data/bc_jja_0.rds")
bc_1 = readRDS("../../data/jja_data/bc_jja_1.rds")
bc_2 = readRDS("../../data/jja_data/bc_jja_2.rds")
bc_3 = readRDS("../../data/jja_data/bc_jja_3.rds")

In [5]:
# Climatología de las obs
ref = climatology(tmx_obs_cel) %>% suppressMessages %>% suppressWarnings

# Climatología del modelos con los 4 leadtime (datos con sesgo)
diff_0 = climatology(tmx_modelo_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_1 = climatology(tmx_modelo_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_2 = climatology(tmx_modelo_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_3 = climatology(tmx_modelo_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Sesgo para cada leadtime
bias_0 = gridArithmetics(diff_0, ref, operator = "-")
bias_1 = gridArithmetics(diff_1, ref, operator = "-")
bias_2 = gridArithmetics(diff_2, ref, operator = "-")
bias_3 = gridArithmetics(diff_3, ref, operator = "-")

# Representación
b_0 = spatialPlot(bias_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(-4, 4, 0.1))
b_1 = spatialPlot(bias_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(-4, 4, 0.1))
b_2 = spatialPlot(bias_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(-4, 4, 0.1))
b_3 = spatialPlot(bias_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(-4, 4, 0.1))

In [6]:
# Climatología de cada leadtime (datos sin sesgo)
diff0 = climatology(bc_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff1 = climatology(bc_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff2 = climatology(bc_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff3 = climatology(bc_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Cálculo del sesgo
bias0 = gridArithmetics(diff0, ref, operator = "-")
bias1 = gridArithmetics(diff1, ref, operator = "-")
bias2 = gridArithmetics(diff2, ref, operator = "-")
bias3 = gridArithmetics(diff3, ref, operator = "-")

# Representación
b0 = spatialPlot(bias0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(-0.004, 0.004, 0.0001))
b1 = spatialPlot(bias1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(-0.004, 0.004, 0.0001))
b2 = spatialPlot(bias2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(-0.004, 0.004, 0.0001))
b3 = spatialPlot(bias3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(-0.004, 0.004, 0.0001))

In [7]:
png("bias_tmax_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(b_0, b_1, b_2, b_3, b0, b1, b2, b3, ncol = 4)
dev.off()

pdf 
  2

## Máscara para los datos

In [8]:
## Calculo el número de días que tmax > 30 grados
nd30_obs = indexGrid(tx = tmx_obs_cel, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings

## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(tmx_obs_cel),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(nd30_obs)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

## Máscara para el moodelo
n.members = getShape(tmx_modelo_0)["member"]
mask.data = mask.bin.spain$Data
mask.4d = array(NA, dim = c(n.members, dim(mask.data)))  # member x time x lat x lon
for (m in 1:n.members) {
    mask.4d[m,,,] = mask.data
}

mask.model = mask.bin.spain  # copia de la estructura
mask.model$Data = mask.4d
attributes(mask.model$Data)$dimensions = c("member", "time", "lat", "lon")

## ndays tmax > 30

### obs

In [ ]:
# Calculo el número de días que tmax > 30 grados
nd30_obs = indexGrid(tx = tmx_obs_cel, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos de ERA5
nd30_obs = gridArithmetics(nd30_obs, mask.bin.spain, operator = "*")

# Computo la climatología
nd_obs = climatology(nd30_obs) %>% suppressMessages %>% suppressWarnings

# Representación
ndobs = spatialPlot(nd_obs, backdrop.theme = "countries", main = "Obs", col.regions = color, at = seq(0, 90, 1))

In [ ]:
png("ndays_tmax30_obs_jja.png", width = 2000, height = 1000, res = 150)
print(ndobs)
dev.off()

### datos modelo raw

In [13]:
# Calculo el número de días que tmax > 30 grados
nd30_0 = indexGrid(tx = tmx_modelo_0, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings
nd30_1 = indexGrid(tx = tmx_modelo_1, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings
nd30_2 = indexGrid(tx = tmx_modelo_2, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings
nd30_3 = indexGrid(tx = tmx_modelo_3, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos del modelo
nd30_0 = gridArithmetics(nd30_0, mask.model, operator = "*")
nd30_1 = gridArithmetics(nd30_1, mask.model, operator = "*")
nd30_2 = gridArithmetics(nd30_2, mask.model, operator = "*")
nd30_3 = gridArithmetics(nd30_3, mask.model, operator = "*")

# Computo la climatología
nd_0 = climatology(nd30_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_1 = climatology(nd30_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_2 = climatology(nd30_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_3 = climatology(nd30_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Representación
nd0 = spatialPlot(nd_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(0, 90, 1))
nd1 = spatialPlot(nd_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(0, 90, 1))
nd2 = spatialPlot(nd_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(0, 90, 1))
nd3 = spatialPlot(nd_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(0, 90, 1))

### datos modelo bC

In [10]:
# Calculo el número de días que tmax > 30 grados
nd30_bc_0 = indexGrid(tx = bc_0, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings
nd30_bc_1 = indexGrid(tx = bc_1, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings
nd30_bc_2 = indexGrid(tx = bc_2, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings
nd30_bc_3 = indexGrid(tx = bc_3, index.code = "TXth", th = 30) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos del modelo
nd30_bc_0 = gridArithmetics(nd30_bc_0, mask.model, operator = "*")
nd30_bc_1 = gridArithmetics(nd30_bc_1, mask.model, operator = "*")
nd30_bc_2 = gridArithmetics(nd30_bc_2, mask.model, operator = "*")
nd30_bc_3 = gridArithmetics(nd30_bc_3, mask.model, operator = "*")

# Computo la climatología
nd_bc_0 = climatology(nd30_bc_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_bc_1 = climatology(nd30_bc_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_bc_2 = climatology(nd30_bc_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_bc_3 = climatology(nd30_bc_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Representación
nd0_bc = spatialPlot(nd_bc_0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(0, 90, 1))
nd1_bc = spatialPlot(nd_bc_1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(0, 90, 1))
nd2_bc = spatialPlot(nd_bc_2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(0, 90, 1))
nd3_bc = spatialPlot(nd_bc_3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(0, 90, 1))

In [14]:
png("ndays_tmax30_model_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(nd0, nd1, nd2, nd3, nd0_bc, nd1_bc, nd2_bc, nd3_bc, ncol = 4)
dev.off()

pdf 
  2

## Bias ndays tmax

### obs y raw

In [17]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_days_0 = veriApply(verifun = "EnsMe", 
                       fcst = nd30_0$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_days_1 = veriApply(verifun = "EnsMe", 
                       fcst = nd30_1$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_days_2 = veriApply(verifun = "EnsMe", 
                       fcst = nd30_2$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_days_3 = veriApply(verifun = "EnsMe", 
                       fcst = nd30_3$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_nd_grid_0 = easyVeri2grid(easyVeri.mat = bias_days_0, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_nd_grid_1 = easyVeri2grid(easyVeri.mat = bias_days_1, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_nd_grid_2 = easyVeri2grid(easyVeri.mat = bias_days_2, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_nd_grid_3 = easyVeri2grid(easyVeri.mat = bias_days_3, obs.grid = tmx_obs_cel, verifun = "EnsMe")

In [18]:
# Representación
bias0 = spatialPlot(climatology(bias_nd_grid_0),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "Raw (lt 0)",
                    at = seq(-40, 40, 1)) %>% suppressMessages %>% suppressWarnings

bias1 = spatialPlot(climatology(bias_nd_grid_1),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "Raw (lt 1)",
                    at = seq(-40, 40, 1)) %>% suppressMessages %>% suppressWarnings

bias2 = spatialPlot(climatology(bias_nd_grid_2),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "Raw (lt 2)",
                    at = seq(-40, 40, 1)) %>% suppressMessages %>% suppressWarnings

bias3 = spatialPlot(climatology(bias_nd_grid_3),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "Raw (lt 3)",
                    at = seq(-40, 40, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [19]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_days_0_bc = veriApply(verifun = "EnsMe", 
                       fcst = nd30_bc_0$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_days_1_bc = veriApply(verifun = "EnsMe", 
                       fcst = nd30_bc_1$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_days_2_bc = veriApply(verifun = "EnsMe", 
                       fcst = nd30_bc_2$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_days_3_bc = veriApply(verifun = "EnsMe", 
                       fcst = nd30_bc_3$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_nd_grid_0_bc = easyVeri2grid(easyVeri.mat = bias_days_0_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_nd_grid_1_bc = easyVeri2grid(easyVeri.mat = bias_days_1_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_nd_grid_2_bc = easyVeri2grid(easyVeri.mat = bias_days_2_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_nd_grid_3_bc = easyVeri2grid(easyVeri.mat = bias_days_3_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")

In [20]:
# Representación
bias0_bc = spatialPlot(climatology(bias_nd_grid_0_bc),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "BC (lt 0)",
                    at = seq(-0.1, 0.1, 0.01)) %>% suppressMessages %>% suppressWarnings

bias1_bc = spatialPlot(climatology(bias_nd_grid_1_bc),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "BC (lt 1)",
                    at = seq(-0.1, 0.1, 0.01)) %>% suppressMessages %>% suppressWarnings

bias2_bc = spatialPlot(climatology(bias_nd_grid_2_bc),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "BC (lt 2)",
                    at = seq(-0.1, 0.1, 0.01)) %>% suppressMessages %>% suppressWarnings

bias3_bc = spatialPlot(climatology(bias_nd_grid_3_bc),
                    backdrop.theme = "countries",
                    col.regions = color,
                    main = "BC (lt 3)",
                    at = seq(-0.1, 0.1, 0.01)) %>% suppressMessages %>% suppressWarnings

In [21]:
png("bias_ndays_tmax30_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(bias0, bias1, bias2, bias3, bias0_bc, bias1_bc, bias2_bc, bias3_bc, ncol = 4)
dev.off()

pdf 
  2

## Correlación ndays tmax

### test de hipótesis para puntos significativos

In [22]:
# Función para calcular correlación de Pearson y valores p entre datos de modelo y observaciones en una grilla espacial
# Además, identifica y marca los puntos con correlación estadísticamente significativa según un umbral de p-valor
#
# Args:
#   model_data: objeto con datos del modelo, estructura esperada con dimensión [miembros, tiempo, latitud, longitud]
#   obs_data: objeto con datos observacionales, estructura con dimensión [tiempo, latitud, longitud]
#   ref_grid: objeto referencia con metadatos espaciales y temporales para construir grillas (xyCoords, Variable, Dates)
#   threshold: umbral para marcar significancia estadística (p-valor), default 0.05
#
# Returns:
#   Lista con:
#     - cor: matriz de correlaciones [lat x lon]
#     - pval: matriz de valores p [lat x lon]
#     - pval_grid: objeto tipo "grid" con valores p y metadatos
#     - pts: lista de objetos para graficar puntos de significancia (stippling)

calc_cor_pval_grid = function(model_data, obs_data, ref_grid, threshold = 0.05) {
    
    # Calcular la media del ensamble para cada punto [tiempo, lat, lon]
    ens_mean = apply(model_data$Data, c(2, 3, 4), mean, na.rm = TRUE)
    
    # Dimensiones espaciales (latitud y longitud)
    lat_n = dim(ens_mean)[2]
    lon_n = dim(ens_mean)[3]
  
    # Inicializar matrices vacías para almacenar correlaciones y p-valores
    cor_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    pval_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    
    # Iterar sobre cada punto espacial
    for (i in 1:lat_n) {
        for (j in 1:lon_n) {
            
            # Extraer series temporales de modelo y observaciones para la celda actual
            pred_series = ens_mean[, i, j]
            obs_series = obs_data$Data[, i, j]
      
            # Filtrar índices con datos completos (no NA)
            valid_idx = complete.cases(pred_series, obs_series)
            
            # Solo calcular correlación si hay suficientes datos (mínimo 10)
            if (sum(valid_idx) >= 10) {
                test = cor.test(pred_series[valid_idx], obs_series[valid_idx], method = "pearson")
                cor_array[i, j] = test$estimate  # Coeficiente de correlación
                pval_array[i, j] = test$p.value  # Valor p de la prueba
            }
        }
    }
  
    # Construir un objeto "grid" para los valores p, con metadatos espaciales y temporales
    pval_grid = list()
    pval_grid$Data = pval_array
    attr(pval_grid$Data, "dimensions") = c("lat", "lon")
    pval_grid$xyCoords = ref_grid$xyCoords
    pval_grid$Variable = ref_grid$Variable
    pval_grid$Dates = ref_grid$Dates
    class(pval_grid) = "grid"

    pval_grid$Variable$varName = "p-values"
    attr(pval_grid$Variable, "description") = "Mapa de p-valores"
    attr(pval_grid$Variable, "units") = ""
    attr(pval_grid$Variable, "longname") = "p-values"
    
    # Crear objetos para graficar puntos de significancia estadística (stippling)
    pts = map.stippling(climatology(pval_grid), 
                        threshold = threshold, 
                        condition = "LT", 
                        pch = 19, col = "black", cex = 0.5) %>% suppressMessages() %>% suppressWarnings()
    
    # Devolver lista con resultados y objetos para plot
    return(list(cor = cor_array, pval = pval_array, pval_grid = pval_grid, pts = pts))
}

In [23]:
# Datos raw
test_cor_0 = calc_cor_pval_grid(nd30_0, nd30_obs, nd30_0)
test_cor_1 = calc_cor_pval_grid(nd30_1, nd30_obs, nd30_0)
test_cor_2 = calc_cor_pval_grid(nd30_2, nd30_obs, nd30_0)
test_cor_3 = calc_cor_pval_grid(nd30_3, nd30_obs, nd30_0)

# Datos bC
test_cor_0_bc = calc_cor_pval_grid(nd30_bc_0, nd30_obs, nd30_0)
test_cor_1_bc = calc_cor_pval_grid(nd30_bc_1, nd30_obs, nd30_0)
test_cor_2_bc = calc_cor_pval_grid(nd30_bc_2, nd30_obs, nd30_0)
test_cor_3_bc = calc_cor_pval_grid(nd30_bc_3, nd30_obs, nd30_0)

Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”


### obs y raw

In [24]:
# Calculo de correlacion para cada leadtime del modelo
cor_days_0 = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_0$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_days_1 = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_1$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_days_2 = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_2$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_days_3 = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_3$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
cor_nd_grid_0 = easyVeri2grid(easyVeri.mat = cor_days_0, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_nd_grid_1 = easyVeri2grid(easyVeri.mat = cor_days_1, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_nd_grid_2 = easyVeri2grid(easyVeri.mat = cor_days_2, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_nd_grid_3 = easyVeri2grid(easyVeri.mat = cor_days_3, obs.grid = tmx_obs_cel, verifun = "EnsCorr")

In [25]:
# Representación
c0 = spatialPlot(climatology(cor_nd_grid_0),
                 backdrop.theme = "countries",
                 sp.layout = list(test_cor_0$pts),
                 col.regions = color,
                 at = seq(-1, 1, 0.1),
                 main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

c1 = spatialPlot(climatology(cor_nd_grid_1),
                 backdrop.theme = "countries",
                 sp.layout = list(test_cor_1$pts),
                 col.regions = color,
                 at = seq(-1, 1, 0.1),
                 main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

c2 = spatialPlot(climatology(cor_nd_grid_2),
                 backdrop.theme = "countries",
                 sp.layout = list(test_cor_2$pts),
                 col.regions = color,
                 at = seq(-1, 1, 0.1),
                 main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

c3 = spatialPlot(climatology(cor_nd_grid_3),
                 backdrop.theme = "countries",
                 sp.layout = list(test_cor_3$pts),
                 col.regions = color,
                 at = seq(-1, 1, 0.1),
                 main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

### obs y bC

In [26]:
# Calculo de correlacion para cada leadtime del modelo
cor_days_0_bc = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_bc_0$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_days_1_bc = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_bc_1$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_days_2_bc = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_bc_2$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_days_3_bc = veriApply(verifun = "EnsCorr", 
                       fcst = nd30_bc_3$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
cor_nd_grid_0_bc = easyVeri2grid(easyVeri.mat = cor_days_0_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_nd_grid_1_bc = easyVeri2grid(easyVeri.mat = cor_days_1_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_nd_grid_2_bc = easyVeri2grid(easyVeri.mat = cor_days_2_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_nd_grid_3_bc = easyVeri2grid(easyVeri.mat = cor_days_3_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")

In [27]:
# Representación
c0_bc = spatialPlot(climatology(cor_nd_grid_0_bc),
                    backdrop.theme = "countries",
                    sp.layout = list(test_cor_0_bc$pts),
                    col.regions = color,
                    at = seq(-1, 1, 0.1),
                    main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

c1_bc = spatialPlot(climatology(cor_nd_grid_1_bc),
                    backdrop.theme = "countries",
                    sp.layout = list(test_cor_1_bc$pts),
                    col.regions = color,
                    at = seq(-1, 1, 0.1),
                    main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

c2_bc = spatialPlot(climatology(cor_nd_grid_2_bc),
                    backdrop.theme = "countries",
                    sp.layout = list(test_cor_2_bc$pts),
                    col.regions = color,
                    at = seq(-1, 1, 0.1),
                    main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

c3_bc = spatialPlot(climatology(cor_nd_grid_3_bc),
                    backdrop.theme = "countries",
                    sp.layout = list(test_cor_3_bc$pts),
                    col.regions = color,
                    at = seq(-1, 1, 0.1),
                    main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [28]:
png("cor_ndays_tmax30_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(c0, c1, c2, c3, c0_bc, c1_bc, c2_bc, c3_bc, ncol = 4)
dev.off()

pdf 
  2

## RMSE ndays tmax

### obs y raw

In [29]:
# Calculo del RMSE para cada leadtime del modelo
rmse_days_0 = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_0$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_days_1 = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_1$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_days_2 = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_2$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_days_3 = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_3$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_nd_grid_0 = easyVeri2grid(easyVeri.mat = rmse_days_0, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_nd_grid_1 = easyVeri2grid(easyVeri.mat = rmse_days_1, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_nd_grid_2 = easyVeri2grid(easyVeri.mat = rmse_days_2, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_nd_grid_3 = easyVeri2grid(easyVeri.mat = rmse_days_3, obs.grid = tmx_obs_cel, verifun = "EnsRmse")

In [30]:
# Representación
rmse_0 = spatialPlot(climatology(rmse_nd_grid_0),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "Raw (lt 0)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

rmse_1 = spatialPlot(climatology(rmse_nd_grid_1),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "Raw (lt 1)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

rmse_2 = spatialPlot(climatology(rmse_nd_grid_2),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "Raw (lt 2)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

rmse_3 = spatialPlot(climatology(rmse_nd_grid_3),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "Raw (lt 3)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [31]:
# Calculo del RMSE para cada leadtime del modelo
rmse_days_0_bc = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_bc_0$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_days_1_bc = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_bc_1$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_days_2_bc = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_bc_2$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_days_3_bc = veriApply(verifun = "EnsRmse", 
                       fcst = nd30_bc_3$Data, 
                       obs = nd30_obs$Data, 
                       ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_nd_grid_0_bc = easyVeri2grid(easyVeri.mat = rmse_days_0_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_nd_grid_1_bc = easyVeri2grid(easyVeri.mat = rmse_days_1_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_nd_grid_2_bc = easyVeri2grid(easyVeri.mat = rmse_days_2_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_nd_grid_3_bc = easyVeri2grid(easyVeri.mat = rmse_days_3_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")

In [32]:
# Representación
rmse_0_bc = spatialPlot(climatology(rmse_nd_grid_0_bc),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "BC (lt 0)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

rmse_1_bc = spatialPlot(climatology(rmse_nd_grid_1_bc),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "BC (lt 1)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

rmse_2_bc = spatialPlot(climatology(rmse_nd_grid_2_bc),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "BC (lt 2)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

rmse_3_bc = spatialPlot(climatology(rmse_nd_grid_3_bc),
                     backdrop.theme = "countries",
                     col.regions = color,
                     main = "BC (lt 3)",
                     at = seq(0, 50, 1)) %>% suppressMessages %>% suppressWarnings

In [33]:
png("rmse_ndays_tmax30_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(rmse_0, rmse_1, rmse_2, rmse_3, rmse_0_bc, rmse_1_bc, rmse_2_bc, rmse_3_bc, ncol = 4)
dev.off()

pdf 
  2

## Ratio varianzas ndays tmax

### obs y raw

In [34]:
# Varianzas, calculo la varianza de cada miembro y luego hago el promedio
var_0 = climatology(nd30_0,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_0 = climatology(var_0,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_1 = climatology(nd30_1,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_1 = climatology(var_1,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_2 = climatology(nd30_2,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_2 = climatology(var_2,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_3 = climatology(nd30_3,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_3 = climatology(var_3,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_obs = climatology(nd30_obs,
                      clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_0 = gridArithmetics(mean_var_0, var_obs, operator = "/")
rv_1 = gridArithmetics(mean_var_1, var_obs, operator = "/")
rv_2 = gridArithmetics(mean_var_2, var_obs, operator = "/")
rv_3 = gridArithmetics(mean_var_3, var_obs, operator = "/")

# Representación
rv0 = spatialPlot(rv_0, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 0)", at = seq(0, 2, 0.1))
rv1 = spatialPlot(rv_1, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 1)", at = seq(0, 2, 0.1))
rv2 = spatialPlot(rv_2, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 2)", at = seq(0, 2, 0.1))
rv3 = spatialPlot(rv_3, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 3)", at = seq(0, 2, 0.1))

### obs y bC

In [35]:
# Varianzas, calculo la varianza de cada miembro y luego hago el promedio
var_0_bc = climatology(nd30_bc_0,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_0_bc = climatology(var_0_bc,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_1_bc = climatology(nd30_bc_1,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_1_bc = climatology(var_1_bc,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_2_bc = climatology(nd30_bc_2,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_2_bc = climatology(var_2_bc,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_3_bc = climatology(nd30_bc_3,
                    clim.fun = list(FUN = "var", na.rm = TRUE),
                    by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_3_bc = climatology(var_3_bc,
                         clim.fun = list(FUN = "mean", na.rm = TRUE),
                         by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_0_bc = gridArithmetics(mean_var_0_bc, var_obs, operator = "/")
rv_1_bc = gridArithmetics(mean_var_1_bc, var_obs, operator = "/")
rv_2_bc = gridArithmetics(mean_var_2_bc, var_obs, operator = "/")
rv_3_bc = gridArithmetics(mean_var_3_bc, var_obs, operator = "/")

# Representación
rv0_bc = spatialPlot(rv_0_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 0)", at = seq(0, 2, 0.1))
rv1_bc = spatialPlot(rv_1_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 1)", at = seq(0, 2, 0.1))
rv2_bc = spatialPlot(rv_2_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 2)", at = seq(0, 2, 0.1))
rv3_bc = spatialPlot(rv_3_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 3)", at = seq(0, 2, 0.1))

In [36]:
png("ratiovars_ndays_tmax30_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(rv0, rv1, rv2, rv3, rv0_bc, rv1_bc, rv2_bc, rv3_bc, ncol = 4)
dev.off()

pdf 
  2

## ROCSS ndays tmax

### Funciones auxiliares

In [37]:
## Función para realizar bootstrap y calcular ROCSS
##
## Esta función calcula el ROC Skill Score (ROCSS) mediante un enfoque bootstrap,
## generando múltiples muestras re-muestreadas con reemplazo sobre la dimensión temporal.
##
## Argumentos:
## - nd_modelo: objeto con los pronósticos originales (formato esperado: [member, time, lat, lon])
## - nd_obs: objeto con las observaciones correspondientes (formato: [time, lat, lon])
## - n_boot: número de muestras bootstrap a generar (por defecto: 1000)
##
## Valor:
## - Lista de 3 arrays (uno por tercil), cada uno con dimensiones [n_boot, lat, lon],
##   que contiene los valores bootstrap del ROCSS para cada tercil.

rocss_bootstrap = function(nd_modelo, nd_obs, n_boot = 1000) {
    
    # Obtener dimensiones del array de pronóstico
    dims = dim(nd_modelo$Data)
    
    # Inicializar lista para guardar ROCSS bootstrap para los tres terciles
    rocss_bootstrap = vector("list", length = 3)
    
    # Extraer observaciones
    obs = nd_obs$Data
    
    # Inicializar arrays vacíos para cada tercil: [n_boot, lat, lon]
    for (i in 1:3) {
        rocss_bootstrap[[i]] = array(NA, dim = c(n_boot, dims[3], dims[4]))
    }
    
    # Bucle principal de bootstrap
    for (b in 1:n_boot) {
        
        # Muestreo con reemplazo sobre la dimensión temporal (índice de tiempo)
        time_idx = sample(1:dims[2], size = dims[2], replace = TRUE)
    
        # Crear pronóstico re-muestreado: [member, time*, sdate, lat, lon]
        fcst_boot = nd_modelo$Data[, time_idx, , , drop = FALSE]
    
        # Calcular ROCSS con veriApply usando la función EnsRocss
        boot_result = veriApply(
            verifun = "EnsRocss",
            fcst = fcst_boot,
            obs = obs,
            prob = c(1/3, 2/3),
            ensdim = 1,   # Dimensión de los miembros del ensamble
            tdim = 2      # Dimensión temporal
        )
    
        # Guardar el resultado para cada tercil en la lista correspondiente
        for (i in 1:3) {
            rocss_bootstrap[[i]][b, , ] = boot_result[[i]]
        }
    }
    
    # Devolver lista con resultados bootstrap por tercil
    return(rocss_bootstrap)
}

In [84]:
## Función para generar capas de "stippling" que indican puntos con ROCSS significativamente mayores que un percentil dado
##
## Parámetros:
## mg               : objeto con datos ROCSS con dimensiones [tercil, member, time, lat, lon]
## rocss_bootstrap  : lista de arrays bootstrap con dimensiones [n_boot, lat, lon] para cada tercil
## ref_grid         : grid de referencia que contiene Dates y xyCoords (coordenadas XY)
## threshold        : percentil del bootstrap para determinar significancia (default 0.95)
##
## Retorna:
## lista de objetos "sp.points" para cada tercil con puntos significativos (stippling)

get_rocss_stippling_layers = function(mg, rocss_bootstrap, ref_grid, threshold = 0.95) {
	
	rocss_orig = mg$Data
	n_terciles = dim(rocss_orig)[1]
	n_lat = dim(rocss_orig)[4]
	n_lon = dim(rocss_orig)[5]
	
	xyCoords = ref_grid$xyCoords
	Dates = ref_grid$Dates
	
	stippling_list = vector("list", n_terciles)
	
	for (t in 1:n_terciles) {
		signif_mask = matrix(NA, nrow = n_lat, ncol = n_lon)
		
		for (i in 1:n_lat) {
			for (j in 1:n_lon) {
				boot_vals = rocss_bootstrap[[t]][, i, j]
				val_orig = rocss_orig[t, 1, 1, i, j]
				if (all(is.na(boot_vals)) || is.na(val_orig)) next
				q95 = quantile(boot_vals, threshold, na.rm = TRUE)
				signif_mask[i, j] = val_orig > q95
			}
		}
		
		# Construcción manual del objeto grid para la capa de significancia
		pval_grid = list()
		pval_grid$Data = signif_mask
		attr(pval_grid$Data, "dimensions") = c("lat", "lon")
		pval_grid$Dates = Dates
		pval_grid$xyCoords = xyCoords
		pval_grid$Variable = list(varName = paste0("significance_tercil_", t))
		class(pval_grid) = "grid"
		
		# Genera puntos de significancia con map.stippling (pch=19, color negro, tamaño 0.5)
		stippling_list[[t]] = suppressWarnings(
			suppressMessages(
				map.stippling(climatology(pval_grid), threshold = 0.5, condition = "GT",
				              pch = 19, col = "black", cex = 0.5)
			)
		)
	}
	
	# Elimina entradas NULL (si algún tercil no tiene puntos significativos)
	stippling_list = Filter(Negate(is.null), stippling_list)
	
	# Filtra para quedarse solo con listas que son puntos espaciales (sp.points)
	stippling_list = Filter(function(x) {
		is.list(x) && length(x) > 1 && x[[1]] == "sp.points"
	}, stippling_list)
	
	return(stippling_list)
}

### obs y raw

In [74]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_days_0 = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_0$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_days_1 = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_1$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_days_2 = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_2$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_days_3 = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_3$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_0 = makeMultiGrid(lapply(rocss_days_0[1:3], "easyVeri2grid", nd30_obs))
mg_1 = makeMultiGrid(lapply(rocss_days_1[1:3], "easyVeri2grid", nd30_obs))
mg_2 = makeMultiGrid(lapply(rocss_days_2[1:3], "easyVeri2grid", nd30_obs))
mg_3 = makeMultiGrid(lapply(rocss_days_3[1:3], "easyVeri2grid", nd30_obs))

In [30]:
set.seed(123)  # Reproducibilidad

# Aplico función de bootstraping
rocss_bootstrap_0 = rocss_bootstrap(nd30_0, nd30_obs)
rocss_bootstrap_1 = rocss_bootstrap(nd30_1, nd30_obs)
rocss_bootstrap_2 = rocss_bootstrap(nd30_2, nd30_obs)
rocss_bootstrap_3 = rocss_bootstrap(nd30_3, nd30_obs)

In [68]:
saveRDS(rocss_bootstrap_0, file = "rocss_bootstrap_raw_jja_0.rds")
saveRDS(rocss_bootstrap_1, file = "rocss_bootstrap_raw_jja_1.rds")
saveRDS(rocss_bootstrap_2, file = "rocss_bootstrap_raw_jja_2.rds")
saveRDS(rocss_bootstrap_3, file = "rocss_bootstrap_raw_jja_3.rds")

In [75]:
rocss_bootstrap_0 = readRDS("../../data/jja_data/rocss_bootstrap_raw_jja_0.rds")
rocss_bootstrap_1 = readRDS("../../data/jja_data/rocss_bootstrap_raw_jja_1.rds")
rocss_bootstrap_2 = readRDS("../../data/jja_data/rocss_bootstrap_raw_jja_2.rds")
rocss_bootstrap_3 = readRDS("../../data/jja_data/rocss_bootstrap_raw_jja_3.rds")

In [76]:
# Aplico función de máscara significativa
pts_layers_0 = get_rocss_stippling_layers(mg_0, rocss_bootstrap_0, nd30_0)
pts_layers_1 = get_rocss_stippling_layers(mg_1, rocss_bootstrap_1, nd30_0)
pts_layers_2 = get_rocss_stippling_layers(mg_2, rocss_bootstrap_2, nd30_0)
pts_layers_3 = get_rocss_stippling_layers(mg_3, rocss_bootstrap_3, nd30_0)

In [77]:
rocss0 = spatialPlot(climatology(mg_0),
                     backdrop.theme = "countries",
                     names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                     layout = c(3,1),
                     col.regions = color,
                     at = seq(-1, 1, 0.05),
                     sp.layout = list(pts_layers_0),
                     main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss1 = spatialPlot(climatology(mg_1),
                     backdrop.theme = "countries",
                     names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                     layout = c(3,1),
                     col.regions = color,
                     at = seq(-1, 1, 0.05),
                     sp.layout = list(pts_layers_1),
                     main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss2 = spatialPlot(climatology(mg_2),
                     backdrop.theme = "countries",
                     names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                     layout = c(3,1),
                     col.regions = color,
                     at = seq(-1, 1, 0.05),
                     sp.layout = list(pts_layers_2),
                     main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss3 = spatialPlot(climatology(mg_3),
                     backdrop.theme = "countries",
                     names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                     layout = c(3,1),
                     col.regions = color,
                     at = seq(-1, 1, 0.05),
                     sp.layout = list(pts_layers_3),
                     main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

In [78]:
png("rocss_ndays_tmax30_raw_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss0, rocss1, rocss2, rocss3, ncol = 2)
dev.off()

pdf 
  2

### obs y bC

In [79]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_days_0_bc = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_bc_0$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_days_1_bc = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_bc_1$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_days_2_bc = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_bc_2$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_days_3_bc = veriApply(verifun = "EnsRocss", 
                         fcst = nd30_bc_3$Data, 
                         obs = nd30_obs$Data,
                         prob = c(1/3, 2/3),
                         ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_0_bc = makeMultiGrid(lapply(rocss_days_0_bc[1:3], "easyVeri2grid", nd30_obs))
mg_1_bc = makeMultiGrid(lapply(rocss_days_1_bc[1:3], "easyVeri2grid", nd30_obs))
mg_2_bc = makeMultiGrid(lapply(rocss_days_2_bc[1:3], "easyVeri2grid", nd30_obs))
mg_3_bc = makeMultiGrid(lapply(rocss_days_3_bc[1:3], "easyVeri2grid", nd30_obs))

In [33]:
set.seed(123)  # Reproducibilidad

# Aplico función de bootstraping
rocss_bootstrap_0_bc = rocss_bootstrap(nd30_bc_0, nd30_obs)
rocss_bootstrap_1_bc = rocss_bootstrap(nd30_bc_1, nd30_obs)
rocss_bootstrap_2_bc = rocss_bootstrap(nd30_bc_2, nd30_obs)
rocss_bootstrap_3_bc = rocss_bootstrap(nd30_bc_3, nd30_obs)

In [69]:
saveRDS(rocss_bootstrap_0_bc, file = "rocss_bootstrap_bc_jja_0.rds")
saveRDS(rocss_bootstrap_1_bc, file = "rocss_bootstrap_bc_jja_1.rds")
saveRDS(rocss_bootstrap_2_bc, file = "rocss_bootstrap_bc_jja_2.rds")
saveRDS(rocss_bootstrap_3_bc, file = "rocss_bootstrap_bc_jja_3.rds")

In [80]:
rocss_bootstrap_0_bc = readRDS("../../data/jja_data/rocss_bootstrap_bc_jja_0.rds")
rocss_bootstrap_1_bc = readRDS("../../data/jja_data/rocss_bootstrap_bc_jja_1.rds")
rocss_bootstrap_2_bc = readRDS("../../data/jja_data/rocss_bootstrap_bc_jja_2.rds")
rocss_bootstrap_3_bc = readRDS("../../data/jja_data/rocss_bootstrap_bc_jja_3.rds")

In [81]:
# Aplico función de máscara significativa
pts_layers_0_bc = get_rocss_stippling_layers(mg_0_bc, rocss_bootstrap_0_bc, nd30_0)
pts_layers_1_bc = get_rocss_stippling_layers(mg_1_bc, rocss_bootstrap_1_bc, nd30_0)
pts_layers_2_bc = get_rocss_stippling_layers(mg_2_bc, rocss_bootstrap_2_bc, nd30_0)
pts_layers_3_bc = get_rocss_stippling_layers(mg_3_bc, rocss_bootstrap_3_bc, nd30_0)

In [82]:
rocss0_bc = spatialPlot(climatology(mg_0_bc),
                        backdrop.theme = "countries",
                        names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                        layout = c(3,1),
                        col.regions = color,
                        at = seq(-1, 1, 0.05),
                        sp.layout = list(pts_layers_0_bc),
                        main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss1_bc = spatialPlot(climatology(mg_1_bc),
                        backdrop.theme = "countries",
                        names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                        layout = c(3,1),
                        col.regions = color,
                        at = seq(-1, 1, 0.05),
                        sp.layout = list(pts_layers_1_bc),
                        main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss2_bc = spatialPlot(climatology(mg_2_bc),
                        backdrop.theme = "countries",
                        names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                        layout = c(3,1),
                        col.regions = color,
                        at = seq(-1, 1, 0.05),
                        sp.layout = list(pts_layers_2_bc),
                        main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss3_bc = spatialPlot(climatology(mg_3_bc),
                        backdrop.theme = "countries",
                        names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                        layout = c(3,1),
                        col.regions = color,
                        at = seq(-1, 1, 0.05),
                        sp.layout = list(pts_layers_3_bc),
                        main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [83]:
png("rocss_ndays_tmax30_bc_jja.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss0_bc, rocss1_bc, rocss2_bc, rocss3_bc, ncol = 2)
dev.off()

pdf 
  2